## **Installing LibAUC**


In [1]:
! pip install medmnist
! pip install libauc==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=6d9afd89e4c34644fb33acf05d27db80f19cad9a5a13fbcf7d2fb13f1410b89c
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.4 MB/s eta 0:00:00


# **Importing Libraries**


In [2]:
import os
import time
from PIL import Image
import numpy as np
import random

from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG
from libauc.models import resnet20 as ResNet20
from libauc.models import resnet18 as ResNet18
from libauc.utils import ImbalancedDataGenerator
from libauc.sampler import DualSampler
from libauc.metrics import auc_roc_score

import medmnist
from medmnist import BreastMNIST
from medmnist import INFO, Evaluator

import torch 
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

## **Reproducibility**



In [3]:
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## **Image Dataset**





In [4]:
class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=32, crop_size=28, mode='train'):
       self.images = images.astype(np.uint8)
       self.targets = targets
       self.mode = mode
       self.transform_train = transforms.Compose([                                                
                              transforms.ToTensor(),
                              # transforms.RandomCrop((crop_size, crop_size), padding=None),
                              transforms.RandomHorizontalFlip(),
                              transforms.Resize((image_size, image_size)),
                              transforms.Normalize(mean=[.5], std=[.5])
                              ])
       self.transform_test = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Resize((image_size, image_size)),
                             transforms.Normalize(mean=[.5], std=[.5])
                              ])
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        image = Image.fromarray(image.astype('uint8'))
        if self.mode == 'train':
            image = self.transform_train(image)
        else:
            image = self.transform_test(image)
        return image, target

# **Paramaters**

In [14]:
# HyperParameters
imratio = 0.1 # for demo 
total_epochs = 200
decay_epochs = [50, 80]
sampling_rate = 0.5
lr = 0.1
margin = 1.5
epoch_decay = 0.03
weight_decay = 0.0001
BATCH_SIZE=128
momentum=0.9

# **Loading datasets**

In [15]:
dataflag = 'breastmnist'
info = INFO[dataflag]
task = info['task']
n_classes = len(info['label'])
info

{'python_class': 'BreastMNIST',
 'description': 'The BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign, and malignant. As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images of 1×500×500 are resized into 1×28×28.',
 'url': 'https://zenodo.org/record/6496656/files/breastmnist.npz?download=1',
 'MD5': '750601b1f35ba3300ea97c75c52ff8f6',
 'task': 'binary-class',
 'label': {'0': 'malignant', '1': 'normal, benign'},
 'n_channels': 1,
 'n_samples': {'train': 546, 'val': 78, 'test': 156},
 'license': 'CC BY 4.0'}

In [16]:
# Load the train dataset
train_dataset = BreastMNIST(root='./', split='train', download=True)
train_images = train_dataset.imgs 
train_labels = train_dataset.labels[:, 0]

# Load the val dataset
val_dataset = BreastMNIST(root='./', split='val', download=True)
val_images = val_dataset.imgs 
val_labels = val_dataset.labels[:, 0]

# Load the test dataset
test_dataset = BreastMNIST(root='./', split='test', download=True)
test_images = test_dataset.imgs 
test_labels = test_dataset.labels[:, 0]

Using downloaded and verified file: ./breastmnist.npz
Using downloaded and verified file: ./breastmnist.npz
Using downloaded and verified file: ./breastmnist.npz


In [17]:
# data augmentations 
trainSet = ImageDataset(train_images, train_labels)
trainSet_eval = ImageDataset(val_images, val_labels, mode='test')
testSet = ImageDataset(test_images, test_labels, mode='test')

# dataloaders
sampler = DualSampler(trainSet, BATCH_SIZE, sampling_rate=sampling_rate)
train_loader = torch.utils.data.DataLoader(trainSet, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2)
val_loader = torch.utils.data.DataLoader(trainSet_eval, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(testSet, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# **Creating models & AUC Optimizer**

In [18]:
# Creating the model
model = ResNet18(pretrained=False, last_activation=None) 
model = model.cuda()

# Defining Loss and Optimizers
loss_fn = AUCMLoss()
optimizer = PESG(model, 
                 loss_fn=loss_fn,
                 lr=lr, 
                 momentum=momentum,
                 margin=margin, 
                 epoch_decay=epoch_decay, 
                 weight_decay=weight_decay)

# **Training**

In [19]:
print ('Start Training')
print ('-'*30)

best_val_auc = 0
best_model = model

train_log = []

for epoch in range(total_epochs):
     if epoch in decay_epochs:
         optimizer.update_regularizer(decay_factor=10) # decrease learning rate by 10x & update regularizer
   
     # TRAINING   
     train_loss = []
     model.train()    
     for train_data, train_targets in train_loader:
         train_data, train_targets  = train_data.cuda(), train_targets.cuda()
         train_data = train_data.expand(-1, 3, -1, -1)
         y_pred = model(train_data)
         y_pred = torch.sigmoid(y_pred)
         loss = loss_fn(y_pred, train_targets)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
         train_loss.append(loss.item())
    
     epoch_loss = np.mean(train_loss)

     # VALIDATION
     with torch.no_grad():
        model.eval()
        val_pred_list = []
        val_true_list = []
        for val_data, val_targets in val_loader:
            val_data  = val_data.cuda()
            val_data = val_data.expand(-1, 3, -1, -1)
            val_pred = model(val_data)
            val_pred_list.append(val_pred.cpu().detach().numpy())
            val_true_list.append(val_targets.numpy())
        val_true = np.concatenate(val_true_list)
        val_pred = np.concatenate(val_pred_list)
        val_auc = auc_roc_score(val_true, val_pred)

        val_pred_binary = (val_pred > 0.5).astype(int)
        val_accuracy = accuracy_score(val_true, val_pred_binary)
          
        if best_val_auc < val_auc:
          best_val_auc = val_auc
          best_model = model

        train_log.append(val_auc)    

     print("epoch: %s, epoch_loss: %.4f, val_auc: %.4f, lr: %.4f, best_val_auc: %.4f"%(epoch, epoch_loss, val_auc, optimizer.lr, best_val_auc ))    

Start Training
------------------------------
epoch: 0, epoch_loss: 0.2070, val_auc: 0.4227, lr: 0.1000, best_val_auc: 0.4227
epoch: 1, epoch_loss: 0.2104, val_auc: 0.5480, lr: 0.1000, best_val_auc: 0.5480
epoch: 2, epoch_loss: 0.1136, val_auc: 0.7018, lr: 0.1000, best_val_auc: 0.7018
epoch: 3, epoch_loss: 0.0479, val_auc: 0.7352, lr: 0.1000, best_val_auc: 0.7352
epoch: 4, epoch_loss: -0.0065, val_auc: 0.8287, lr: 0.1000, best_val_auc: 0.8287
epoch: 5, epoch_loss: -0.0639, val_auc: 0.8162, lr: 0.1000, best_val_auc: 0.8287
epoch: 6, epoch_loss: -0.0552, val_auc: 0.8170, lr: 0.1000, best_val_auc: 0.8287
epoch: 7, epoch_loss: -0.0346, val_auc: 0.8363, lr: 0.1000, best_val_auc: 0.8363
epoch: 8, epoch_loss: -0.0773, val_auc: 0.5940, lr: 0.1000, best_val_auc: 0.8363
epoch: 9, epoch_loss: -0.0535, val_auc: 0.8964, lr: 0.1000, best_val_auc: 0.8964
epoch: 10, epoch_loss: -0.0872, val_auc: 0.8814, lr: 0.1000, best_val_auc: 0.8964
epoch: 11, epoch_loss: -0.0560, val_auc: 0.8764, lr: 0.1000, best_

# **Testing**

In [20]:
# Evaluation on Test data

test_pred_list = []
test_true_list = [] 
for test_data, test_targets in test_loader:
    test_data  = test_data.cuda()
    test_data = test_data.expand(-1, 3, -1, -1)
    test_pred = best_model(test_data)
    test_pred_list.append(test_pred.cpu().detach().numpy())
    test_true_list.append(test_targets.numpy())
test_true = np.concatenate(test_true_list)
test_pred = np.concatenate(test_pred_list)
test_auc =  auc_roc_score(test_true, test_pred) 

test_pred_binary = (test_pred > 0.5).astype(int)
test_accuracy = accuracy_score(test_true, test_pred_binary)
print("Test AUC: %.4f, test_accuracy: %.4f"%(test_auc, test_accuracy))      

Test AUC: 0.9012, test_accuracy: 0.8718


In [21]:
# Saving the best model
state = {
    'net': best_model.state_dict(),
}

output_root = os.path.join('./output', dataflag)
if not os.path.exists(output_root):
    os.makedirs(output_root)

filename = dataflag + '_auc_' + str(round(test_auc,4)) + '_model.pth'
path = os.path.join(output_root, filename)
torch.save(state, path)

In [22]:
# Eval function similar to demo file

def evaluate(net, test_loader):
    # Testing AUC
    score_list = list()
    label_list = list()
    for tmp_data, tmp_label in test_loader:
        tmp_data, tmp_label = tmp_data.cuda(), tmp_label.cuda()
        tmp_data = tmp_data.expand(-1, 3, -1, -1)        
        tmp_score = net(tmp_data).detach().clone().cpu()
        score_list.append(tmp_score)
        label_list.append(tmp_label.cpu())
    test_label = torch.cat(label_list)
    test_score = torch.cat(score_list)
                   
    test_auc = metrics.roc_auc_score(test_label, test_score)                   
    print("Test: %.4f"%test_auc, flush=True)

checkpoint = torch.load(path)
final_model = ResNet18() 
final_model = final_model.cuda()
final_model.load_state_dict(checkpoint['net'])
evaluate(final_model, test_loader)

Test: 0.9058
